In [1]:
import tensorflow as tf
from sklearn.datasets import load_boston
boston_housing = tf.keras.datasets.boston_housing

(x_train, y_train),(x_test, y_test) = boston_housing.load_data()

In [2]:
boston = load_boston()

In [4]:
import pandas as pd

x_train = pd.DataFrame(x_train, columns=list(boston.feature_names))
y_train = pd.Series(y_train)
x_test = pd.DataFrame(x_test, columns=list(boston.feature_names))
y_test = pd.Series(y_test)

In [6]:
from tensorflow.keras import Model, Input
from tensorflow.keras.backend import shape
from tensorflow.keras.layers import Dense, concatenate

def build_model():
    inputs = {col : Input(name=col,shape=(1,), dtype='float32') for col in list(boston.feature_names)}
    
    x = concatenate([inputs[col] for col in list(boston.feature_names)])
    x = Dense(20,activation='relu',name='hidden1')(x)
    out = Dense(1)(x)

    model = Model(inputs=inputs,outputs=out)

    model.compile(optimizer='adam', 
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model



inp = (dict(x_train), y_train)
inp_ds = tf.data.Dataset.from_tensor_slices(inp).batch(32)
val = (dict(x_test), y_test)
val_ds = tf.data.Dataset.from_tensor_slices(val).batch(32)

model = build_model()

model.fit(inp_ds, epochs=5, validation_data=val_ds)

Train for 13 steps, validate for 4 steps
Epoch 1/5
13/13 [==============================] - 1s 107ms/step - loss: 1551.6971 - mae: 34.9814 - mse: 1560.7083 - val_loss: 1156.4446 - val_mae: 30.6870 - val_mse: 1211.4019
Epoch 2/5
13/13 [==============================] - 0s 4ms/step - loss: 959.1615 - mae: 27.1919 - mse: 967.3128 - val_loss: 671.8089 - val_mae: 23.3102 - val_mse: 700.1531
Epoch 3/5
13/13 [==============================] - 0s 5ms/step - loss: 592.2898 - mae: 20.8860 - mse: 598.1719 - val_loss: 429.3327 - val_mae: 17.8446 - val_mse: 433.1288
Epoch 4/5
13/13 [==============================] - 0s 4ms/step - loss: 395.4277 - mae: 16.5794 - mse: 399.3449 - val_loss: 300.7395 - val_mae: 14.4639 - val_mse: 292.1638
Epoch 5/5
13/13 [==============================] - 0s 5ms/step - loss: 299.9066 - mae: 14.1779 - mse: 302.6371 - val_loss: 234.0486 -

In [7]:
model.predict(list(dict(x_train).values()))

array([[157.51585 ],
       [181.38756 ],
       [339.73233 ],
       [ 99.586006],
       [339.40668 ],
       [144.6726  ],
       [344.2741  ],
       [214.25212 ],
       [199.73387 ],
       [229.6666  ],
       [339.04825 ],
       [247.19365 ],
       [151.51192 ],
       [329.46335 ],
       [ 92.84889 ],
       [245.51001 ],
       [342.4381  ],
       [186.54414 ],
       [185.06339 ],
       [202.34619 ],
       [238.88588 ],
       [124.91912 ],
       [207.51947 ],
       [151.31535 ],
       [103.697815],
       [344.13373 ],
       [160.40355 ],
       [116.144844],
       [160.15099 ],
       [203.959   ],
       [152.40332 ],
       [338.73776 ],
       [305.81192 ],
       [149.12926 ],
       [124.42384 ],
       [139.16783 ],
       [165.2032  ],
       [162.04352 ],
       [140.50412 ],
       [208.58273 ],
       [345.87195 ],
       [100.254234],
       [101.69576 ],
       [324.10797 ],
       [342.98022 ],
       [158.64494 ],
       [231.24554 ],
       [159.9

In [12]:
from interpret.ext.blackbox import TabularExplainer
explainer = TabularExplainer(model, x_train, features=boston.feature_names)

global_explanation = explainer.explain_global(x_test, true_labels=y_test)

ValueError: Could not find valid explainer to explain model